In [ ]:
%%capture

!pip install -U torch
!pip install -U transformers
!pip install -U sentencepiece
!pip install -U tqdm

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch, sentencepiece
from tqdm.auto import tqdm

In [ ]:
# checking the  GPU allocation

!nvidia-smi

Tue Jul 13 11:38:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pprint as pp

print("Summarized text is :\n")


Summarized text is :



In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = [
    """ """
]

model_name = ""  #give input model of text here
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(
    device
)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

del batch, translated, model, tokenizer  # changing below

In [ ]:
import os, shutil
from google.colab import files

# upload zip file. 
in_archive = files.upload()
the_wd = os.getcwd()
src_dir = os.path.join(the_wd, "input_txt_files")  # put the files here
os.makedirs(src_dir, exist_ok=True)
shutil.unpack_archive(list(in_archive.keys())[0], src_dir)

In [ ]:
from natsort import natsorted
from os import listdir
from os.path import join, basename, isfile

req_ext = ".txt"
appr_files = [
    join(src_dir, f)
    for f in listdir(src_dir)
    if isfile(join(src_dir, f)) and f.endswith(req_ext)
]
print("preview of {} loaded text files \n".format(len(appr_files)))
n_preview = 3
pp.pprint(appr_files[:n_preview], compact=True, indent=5)

In [ ]:
out_dir = os.path.join(the_wd, "")  
os.makedirs(out_dir, exist_ok=True)

model_name = ""
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
summarizations = []
summarization_percentages = []

for tfile in tqdm(appr_files, total=len(appr_files), desc="summarizing input files"):

    torch.cuda.empty_cache()  # so GPU doesn't crash
    with open(tfile, mode="r", encoding="utf-8", errors="ignore") as src_f:
        this_text = src_f.read()

    batch = tokenizer(
        this_text, truncation=True, padding="longest", return_tensors="pt"
    ).to(device)
    translated = model.generate(**batch, no_repeat_ngram_size=2)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    summarizations.append(tgt_text)

    this_outname = "SUM_" + basename(tfile)
    with open(
        join(out_dir, this_outname), mode="w", encoding="utf-8", errors="ignore"
    ) as out_f:
        out_f.writelines(tgt_text)
    # metric comp.
    summarization_percentages.append(round(100 * len(tgt_text) / len(this_text), 4))
    del batch, translated, tgt_text  # so GPU doesn't crash

print(
    "Files were summarized down to the following percentages:\n",
    summarization_percentages,
)  # by ratio of characters in text

In [ ]:
# display some results
import random

n_preview = 5
print("printout of {} of the summary files \n".format(n_preview))
pp.pprint(random.sample(summarizations, k=n_preview), indent=5)

In [ ]:
download_output_files = True  # @param {type:"boolean"}

In [ ]:
# create zip archive and download
from datetime import datetime

zip_dir = join(the_wd, "zipped_outputs")
os.makedirs(zip_dir, exist_ok=True)
summary_header = "summary_file_archive_" + datetime.now().strftime("%d%m%Y")
shutil.make_archive(join(zip_dir, summary_header), "zip", out_dir)

if download_output_files:
    files.download(join(zip_dir, summary_header + ".zip"))
    print("downloaded files - ", datetime.now())
else:
    print("download_output_files is set to: ", download_output_files)